# SpotifyDB Notebook
This notebook demonstrates setting up and using the SpotifyDB utilities.

In [ ]:
#in powershell
# %pip install spotipy duckdb python-dotenv pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## create cred.py  in the root directory 
add your own client id and secrets using the values you have optained from spotify.


### cred.py
```python
# get values from 
# https://developer.spotify.com/dashboard/
spotify_client_id = "your id"
spotify_client_secret = "your secret"
```


In [2]:
# import the credentials from cred.py
from cred import spotify_client_id, spotify_client_secret

## import libraries

In [ ]:
import os, time
# from dotenv import load_dotenv

import duckdb
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyOAuth

# custom functions
import functions as fn

from IPython.display import display

## setting up the environment, api, and duckdb

In [4]:
# Load .env
# load_dotenv()

CLIENT_ID = spotify_client_id
CLIENT_SECRET = spotify_client_secret
REDIRECT_URI = "http://127.0.0.1:8000/callback"
SCOPE = (
    "user-library-read user-read-recently-played user-top-read user-read-playback-state user-follow-read playlist-read-private playlist-modify-private playlist-modify-public"
)

assert CLIENT_ID and CLIENT_SECRET and REDIRECT_URI, "Missing Spotify env vars"

sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET,
        redirect_uri=REDIRECT_URI,
        scope=SCOPE,
        open_browser=True,  # will open auth page in browser
        cache_path=".cache-spotifydb"  # token cache
    )
)

current_user = sp.current_user()
current_user["display_name"], current_user["id"]


('1257524228', '1257524228')

In [5]:
# In-memory DB (great for playing around)
# con = duckdb.connect(database=':memory:')

# If you want persistent on-disk:
con = duckdb.connect(database='spotify.duckdb')


In [ ]:
df_tables = con.execute(f"SHOW TABLES").df()
display(df_tables)


,name
0,AI_Covers
1,All_American_Rejects
2,Blink182
3,Covers
4,MagnoliaPark
5,NTS_Covers
6,Sum41
7,TheParadox
8,followed_artist
9,my_liked_songs


## function wrapper 

In [ ]:
def get_item(table_name:str,source_type:str, source_id:str,verbose=True):
    """
    source_type: playlist, album, artist, liked_songs, top_tracks, recently_played
    """
    global con
    global sp

    table_age = fn.duckdb_table_age(con, table_name)    
    print(f"table age: {table_age} days old")

    if table_age is None or table_age > 1.0:

        if source_type == "playlist":
            items = fn.load_tracks_from_playlist(sp, source_id)
        elif source_type == "album":
            items = fn.load_tracks_from_album(sp, source_id)
        elif source_type == "artist":
            items = fn.load_tracks_from_artist(sp, source_id)
        elif source_type == "liked_songs":
            items = fn.load_my_saved_tracks(sp)
        elif source_type == "top_tracks":
            items = fn.load_my_top_tracks(sp)
        elif source_type == "recently_played":
            # recently_played tracks 
            items = fn.load_my_recently_played(sp)
        # elif source_type == "recently_played_last3M":
        #     # recently played tracks in the last 3 months
        #     now = int(time.time() * 1000)
        #     three_months_ago = now - (90 * 24 * 60 * 60 * 1000)
        #     items = fn.load_my_recently_played(sp, after=three_months_ago)
        # elif source_type == "one_year_ago":
        #     # stuff i haven't played in over a year
        #     now = int(time.time() * 1000)
        #     one_year_ago = now - (365 * 24 * 60 * 60 * 1000)
        #     items = fn.load_my_recently_played(sp, before=one_year_ago)
        else:
            raise ValueError(f"Unknown source_type: {source_type}")
        
        try:
            fn.df_to_duckdb(con, items, table_name)
        except Exception as e:
            print(f"Error saving to DuckDB: {e}")
    else:
        items = fn.duckdb_to_df(con, table_name)
    
    if verbose:
        display(f"{table_name}: {len(items)} records")
        display(items.head())

    return items

    

## geting basic data 

### my followed artist 

In [ ]:


table_age = fn.duckdb_table_age(con, "followed_artist")

print(f"table age: {table_age} days old")


if table_age is None or table_age > 1.0:
    print("Refreshing followed_artist table...")

    followed_artist = fn.get_followed_artists_df(sp)
    display(followed_artist.head())

    print("Loading followed_artist table from DuckDB...")
    fn.df_to_duckdb(con, followed_artist, "followed_artist")
else:
    followed_artist = fn.duckdb_to_df(con, "followed_artist")
    print("Loaded followed_artist table from DuckDB.")


display(f"Followed Artists: {len(followed_artist)} records")
display(followed_artist.head())


table age: 0.6879340560651488 days old
Loaded followed_artist table from DuckDB.


'Followed Artists: 48 records'

,artist_id,name,followers,popularity,genres,uri
0,0LM6g6DH6tKLjDAjLYIwiD,Kolesterol,16,0,,spotify:artist:0LM6g6DH6tKLjDAjLYIwiD
1,7I95CM75shzCjHuTzrepjM,Nova Twins,184469,52,,spotify:artist:7I95CM75shzCjHuTzrepjM
2,2qybVy8UqYXDx6x6BZedPE,Shadow Realm,371,11,,spotify:artist:2qybVy8UqYXDx6x6BZedPE
3,2mKV7sPvlBvzyPLeZhzT6q,Skyebrows,703,27,,spotify:artist:2mKV7sPvlBvzyPLeZhzT6q
4,2gTWLwanLiNZFR0iNPD847,EZ Band,120737,44,"latin country, tejano, norteño",spotify:artist:2gTWLwanLiNZFR0iNPD847


### my liked songs

In [9]:
my_liked_songs = get_item("my_liked_songs", "liked_songs", None)


table age: 0.6878237802783648 days old


'my_liked_songs: 1737 records'

,saved_at,track_id,track_name,album_name,album_id,artist_name,artist_id,duration_ms,explicit,popularity,is_local,uri
0,2025-12-25T01:28:59Z,1RKvLR24HvA3MrGXVGUEvI,Too Old to Rage,Too Old to Rage,4mhEJ8BVDZTsHUXAREubK3,Kolesterol,0LM6g6DH6tKLjDAjLYIwiD,93960,False,0,False,spotify:track:1RKvLR24HvA3MrGXVGUEvI
1,2025-12-09T20:32:49Z,3LERinY2BE29HbccnJSQBW,Th Real Slim Shady (1950's) Motown Soul/Blues,Th Real Slim Shady (1950's) Motown Soul/Blues,6eiOLn9MxT98cB3JYC4S3A,IAXMUSIC,1GlWzEuSxQe13cfCZRDYEw,333479,True,42,False,spotify:track:3LERinY2BE29HbccnJSQBW
2,2025-12-07T14:37:58Z,69qPipyf6iRMPmnkBDGSLp,Thong Song (Motown Choir),Thong Song (Motown Choir),32wGCu2YCK4lfgdijS3lZB,"gino, 1950s Orchestra",3M22l9h7nL6oPStpf5AK6p,180961,True,42,False,spotify:track:69qPipyf6iRMPmnkBDGSLp
3,2025-12-07T14:34:30Z,4GgBpJYckh7S5Hr0OEmupd,Thong Song,Unleash The Dragon,77s7LHeb6Z7gDXWqCkyFrp,Sisqo,6x9QLdzo6eBZxJ1bHsDkjg,253733,False,52,False,spotify:track:4GgBpJYckh7S5Hr0OEmupd
4,2025-12-02T21:16:26Z,3WwoUlqRzqd3fAnMH8CXWJ,Joe's Rogaine Experience,Joe's Rogaine Experience,5vFGbEPRn8lsNLqpkvvuPP,Skyebrows,2mKV7sPvlBvzyPLeZhzT6q,264192,False,32,False,spotify:track:3WwoUlqRzqd3fAnMH8CXWJ


## new_liked_songs

In [ ]:
new_liked_songs = duckdb.query(
"""
select * from my_liked_songs 
order by saved_at 
desc limit 50
"""
).to_df()


display(f"new_liked_songs: {len(new_liked_songs)}")
display(new_liked_songs.head())


# let's save this as a Playlist on Spotify
playlist_id = fn.create_playlist(
    sp,
    name="**New Liked Songs",
    description="My 50 most recently liked songs, updated via Spotify API",
    public=True,
    overwrite_if_exists=True
)

fn.add_tracks_to_playlist(sp, playlist_id, new_liked_songs["uri"].tolist())


'new_liked_songs: 100'

,saved_at,track_id,track_name,album_name,album_id,artist_name,artist_id,duration_ms,explicit,popularity,is_local,uri
0,2025-12-25T01:28:59Z,1RKvLR24HvA3MrGXVGUEvI,Too Old to Rage,Too Old to Rage,4mhEJ8BVDZTsHUXAREubK3,Kolesterol,0LM6g6DH6tKLjDAjLYIwiD,93960,False,0,False,spotify:track:1RKvLR24HvA3MrGXVGUEvI
1,2025-12-09T20:32:49Z,3LERinY2BE29HbccnJSQBW,Th Real Slim Shady (1950's) Motown Soul/Blues,Th Real Slim Shady (1950's) Motown Soul/Blues,6eiOLn9MxT98cB3JYC4S3A,IAXMUSIC,1GlWzEuSxQe13cfCZRDYEw,333479,True,42,False,spotify:track:3LERinY2BE29HbccnJSQBW
2,2025-12-07T14:37:58Z,69qPipyf6iRMPmnkBDGSLp,Thong Song (Motown Choir),Thong Song (Motown Choir),32wGCu2YCK4lfgdijS3lZB,"gino, 1950s Orchestra",3M22l9h7nL6oPStpf5AK6p,180961,True,42,False,spotify:track:69qPipyf6iRMPmnkBDGSLp
3,2025-12-07T14:34:30Z,4GgBpJYckh7S5Hr0OEmupd,Thong Song,Unleash The Dragon,77s7LHeb6Z7gDXWqCkyFrp,Sisqo,6x9QLdzo6eBZxJ1bHsDkjg,253733,False,52,False,spotify:track:4GgBpJYckh7S5Hr0OEmupd
4,2025-12-02T21:16:26Z,3WwoUlqRzqd3fAnMH8CXWJ,Joe's Rogaine Experience,Joe's Rogaine Experience,5vFGbEPRn8lsNLqpkvvuPP,Skyebrows,2mKV7sPvlBvzyPLeZhzT6q,264192,False,32,False,spotify:track:3WwoUlqRzqd3fAnMH8CXWJ


## cream of crop

In [ ]:
cream_of_crop = duckdb.query(
"""
select * from my_liked_songs 
order by popularity 
desc limit 100
"""
).to_df()


display(f"cream_of_crop: {len(cream_of_crop)}")
display(cream_of_crop.head())

# let's save this as a Playlist on Spotify
playlist_id = fn.create_playlist(
    sp,
    name="**Cream of Crop",
    description="My 100 most popular liked songs, updated via Spotify API",
    public=True,
    overwrite_if_exists=True
)

fn.add_tracks_to_playlist(sp, playlist_id, cream_of_crop["uri"].tolist())

'cream_of_crop: 100'

,saved_at,track_id,track_name,album_name,album_id,artist_name,artist_id,duration_ms,explicit,popularity,is_local,uri
0,2025-09-03T18:14:36Z,1CPZ5BxNNd0n0nF4Orb9JS,Golden,KPop Demon Hunters (Soundtrack from the Netfli...,14JkAa6IiFaOh5s0nMyMU9,"HUNTR/X, EJAE, AUDREY NUNA, REI AMI, KPop Demo...",2yNNYQBChuox9A5Ka93BIn,194607,False,98,False,spotify:track:1CPZ5BxNNd0n0nF4Orb9JS
1,2019-01-12T18:41:51Z,7BKLCZ1jbUBVqRi2FVlTVw,Closer,Closer,0rSLgV8p5FzfnqlEk4GzxE,"The Chainsmokers, Halsey",69GGBxA162lTqCwzJG5jLp,244960,False,88,False,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw
2,2019-09-21T18:43:08Z,6ECp64rv50XVz93WvxXMGF,This Love,Songs About Jane: 10th Anniversary Edition,5zClcGCSWj926AMjvBNSLc,Maroon 5,04gDigrS5kc9YWfZHwBETP,206200,False,87,False,spotify:track:6ECp64rv50XVz93WvxXMGF
3,2015-07-20T21:23:34Z,57bgtoPSgt236HzfBOd8kj,Thunderstruck,The Razors Edge,4vu7F6h90Br1ZtYYaqfITy,AC/DC,711MCceyCBcFnzjGY4Q7Un,292333,False,87,False,spotify:track:57bgtoPSgt236HzfBOd8kj
4,2015-07-20T21:23:33Z,2zYzyRzz6pRmhPzyfMEC8s,Highway to Hell,Highway to Hell,10v912xgTZbjAtYfyKWJCS,AC/DC,711MCceyCBcFnzjGY4Q7Un,208110,False,87,False,spotify:track:2zYzyRzz6pRmhPzyfMEC8s


## discover_these 

In [ ]:
top_artist = followed_artist.head(10)['artist_id'].to_list()

disc_these = pd.DataFrame()
for artist_id in top_artist:
    artist_albums = fn.get_artist_top_tracks_df(sp, artist_id)
    disc_these = pd.concat([disc_these, artist_albums], ignore_index=True)


display(f"disc_these: {len(disc_these)}")
display(disc_these.head())


# let's save this as a Playlist on Spotify
playlist_id = fn.create_playlist(
    sp,
    name="**discover these",
    description="Best songs from recently followed artist, updated via Spotify API",
    public=True,
    overwrite_if_exists=True
)

fn.add_tracks_to_playlist(sp, playlist_id, disc_these["uri"].tolist())


'disc_these: 79'

,track_id,name,album_name,album_id,artist_name,artist_ids,popularity,duration_ms,explicit,preview_url,uri
0,1RKvLR24HvA3MrGXVGUEvI,Too Old to Rage,Too Old to Rage,4mhEJ8BVDZTsHUXAREubK3,Kolesterol,0LM6g6DH6tKLjDAjLYIwiD,0,93960,False,None,spotify:track:1RKvLR24HvA3MrGXVGUEvI
1,47slGWPq7P8eiKinZFY3Jm,Sleep Tight,I Miss 1999,7uz7lQWIziksh0AW8pj2Y0,Kolesterol,0LM6g6DH6tKLjDAjLYIwiD,0,112343,False,None,spotify:track:47slGWPq7P8eiKinZFY3Jm
2,2Pza97YPP6TqsTkWvsbABB,Buzzkill Parade,I Miss 1999,7uz7lQWIziksh0AW8pj2Y0,Kolesterol,0LM6g6DH6tKLjDAjLYIwiD,0,163392,False,None,spotify:track:2Pza97YPP6TqsTkWvsbABB
3,0MDGZgIhYzA2tgipFLV8tU,I Miss 1999,I Miss 1999,7uz7lQWIziksh0AW8pj2Y0,Kolesterol,0LM6g6DH6tKLjDAjLYIwiD,0,174264,False,None,spotify:track:0MDGZgIhYzA2tgipFLV8tU
4,540RAknNOWBlbsDzriOKtj,Glowdown?,I Miss 1999,7uz7lQWIziksh0AW8pj2Y0,Kolesterol,0LM6g6DH6tKLjDAjLYIwiD,0,114840,False,None,spotify:track:540RAknNOWBlbsDzriOKtj


## let's mix some playlists

### Covers++

In [ ]:

# load some playlists
Covers = get_item("Covers", "playlist", "6jfY6NVENX592ZhLizN4HO",verbose=False)
AI_Covers = get_item("AI_Covers", "playlist", "5xooQuxBYK7ZXN4dhSQ9GL",verbose=False)
NTS_Covers = get_item("NTS_Covers", "playlist", "53pyL7jy1hbFbttiZZ8g1D",verbose=False)

# sort by popularity
Covers = Covers.sort_values(by=['popularity'], ascending=False).reset_index(drop=True)
AI_Covers = AI_Covers.sort_values(by=['popularity'], ascending=False).reset_index(drop=True)
NTS_Covers = NTS_Covers.sort_values(by=['popularity'], ascending=False).reset_index(drop=True)

# combine, drop duplicates, re-sort
CoversPP = pd.concat(
    [
        Covers.head(60), 
        AI_Covers.head(60),
        NTS_Covers.head(20)
    ], 
    ignore_index=True
    )
CoversPP = CoversPP.drop_duplicates()
CoversPP = CoversPP.sort_values(by=['popularity'], ascending=False).reset_index(drop=True)

display(f"CoversPP: {len(CoversPP)}")
display(CoversPP.head())

# save the playlist 
playlist_id = fn.create_playlist(
    sp,
    name="**Covers ++",
    description="Some of the Best Covers from my picks and AI, updated via Spotify API",
    public=True,
    overwrite_if_exists=True
)

fn.add_tracks_to_playlist(sp, playlist_id, CoversPP["uri"].tolist())


table age: 0.6877659564934395 days old
table age: 0.6877313796761964 days old
table age: 0.6877049802905983 days old


'CoversPP: 131'

,source_type,source_id,track_id,name,album_name,album_id,artist_name,artist_ids,popularity,duration_ms,explicit,preview_url,uri
0,6jfY6NVENX592ZhLizN4HO,playlist,1MTQHCpraD4S8g5PAFKzoj,Behind Blue Eyes,Results May Vary,3oUoQ0UH7Rv06x1kpSjS36,Limp Bizkit,165ZgPlLkK7bf5bDoFc6Sb,79,269973,False,<NA>,spotify:track:1MTQHCpraD4S8g5PAFKzoj
1,6jfY6NVENX592ZhLizN4HO,playlist,1OxcIUqVmVYxT6427tbhDW,Fly Away,5,6MCNMOCRsh6nxs7PNzc0zN,Lenny Kravitz,5gznATMVO85ZcLTkE9ULU7,77,221332,False,<NA>,spotify:track:1OxcIUqVmVYxT6427tbhDW
2,6jfY6NVENX592ZhLizN4HO,playlist,5z6xHjCZr7a7AIcy8sPBKy,Smooth Criminal,Anthology,5klPnHQ5dy6Qm2Ul7h1lp5,Alien Ant Farm,6TZdvF1kFzwnQLgHQynzsO,75,209266,False,<NA>,spotify:track:5z6xHjCZr7a7AIcy8sPBKy
3,6jfY6NVENX592ZhLizN4HO,playlist,7uSsHbBFFAnkRQR1rDwP3L,Black Betty - Single Edit,Tonight Alright,5WqHoLszhaCZHgtTebMx8X,Spiderbait,6P7kkhED6EPrfoZuxz20Fo,68,205973,False,<NA>,spotify:track:7uSsHbBFFAnkRQR1rDwP3L
4,6jfY6NVENX592ZhLizN4HO,playlist,5MYsvAONqcsIVOwu2HTQ3p,Faith,"Three Dollar Bill, Y'all $",3I34EprFC0rlAO4pKNeX1t,Limp Bizkit,165ZgPlLkK7bf5bDoFc6Sb,67,146480,True,<NA>,spotify:track:5MYsvAONqcsIVOwu2HTQ3p


#### this is just a more sql way of doing the same thing
* similar output

```python 


# load some playlists
Covers = get_item("Covers", "playlist", "6jfY6NVENX592ZhLizN4HO",verbose=False)
AI_Covers = get_item("AI_Covers", "playlist", "5xooQuxBYK7ZXN4dhSQ9GL",verbose=False)
NTS_Covers = get_item("NTS_Covers", "playlist", "53pyL7jy1hbFbttiZZ8g1D",verbose=False)


# display(NTS_Covers.head())

CoversPP = duckdb.query(
"""

select 
distinct * from 
(

    select * from 
    (
        select * from Covers
        order by popularity desc 
        limit 60
    ) A

    union all 

    select * from     
    (    
        select * from AI_Covers 
        order by popularity desc 
        limit 60
    ) B

    union all

    select * from     
    (    
        select * from NTS_Covers 
        order by popularity desc 
        limit 20
    ) C

) D
order by popularity 
desc limit 150

"""
).to_df()


display(f"CoversPP: {len(CoversPP)}")
display(CoversPP.head())

# save the playlist 
playlist_id = fn.create_playlist(
    sp,
    name="**Covers ++",
    description="Some of the Best Covers from my picks and AI, updated via Spotify API",
    public=True,
    overwrite_if_exists=True
)

fn.add_tracks_to_playlist(sp, playlist_id, CoversPP["uri"].tolist())
```

## forgotten tracks

In [ ]:

recently_played = get_item("recently_played", "recently_played", None, verbose=False)

forgotten_tracks = duckdb.query(
"""

select * from 
(
    select 
    distinct 
    * from my_liked_songs  mls
    left join recently_played rp
    on rp.track_id = mls.track_id
    where rp.track_id is null
    order by saved_at asc --saved a long time ago
    limit 500
)
order by random()
limit 150

"""
).to_df()


display(f"forgotten_tracks: {len(forgotten_tracks)}")
display(forgotten_tracks.head())

# save the playlist 
playlist_id = fn.create_playlist(
    sp,
    name="**Forgotten Tracks",
    description="Some of my liked tracks that i haven't listened to in a while, updated via Spotify API",
    public=True,
    overwrite_if_exists=True
)

fn.add_tracks_to_playlist(sp, playlist_id, CoversPP["uri"].tolist())


table age: 0.6877030019324135 days old


'forgotten_tracks: 150'

,saved_at,track_id,track_name,album_name,album_id,artist_name,artist_id,duration_ms,explicit,popularity,...,album_name_1,album_id_1,artist_name_1,artist_ids,popularity_1,duration_ms_1,explicit_1,preview_url,uri_1,played_at
0,2015-07-20T21:17:35Z,4zpMmlQXeQ5PJXZSI02W99,I Miss You,blink-182 (Explicit Version),7lCGYAUEA2caN3ZYzNZodo,blink-182,6FBDaR13swtiWwGhX1WQsP,227213,False,0,...,None,None,None,None,<NA>,<NA>,<NA>,<NA>,None,None
1,2015-07-20T21:16:28Z,3BcOuqxdsBWxWIYfHlbFlg,Unity,Keep Your Receipt,4AmGy9JOEGUTjyZnstnlAt,Reel Big Fish,3bXhZFreBJF4QDUUiMmtZW,307106,False,6,...,None,None,None,None,<NA>,<NA>,<NA>,<NA>,None,None
2,2015-07-20T21:16:20Z,7B3V59DJ3sdhXGG05itbpo,Brand New Song,Why Do They Rock So Hard,77ONgv2XviPXVVQo7kFfmA,Reel Big Fish,3bXhZFreBJF4QDUUiMmtZW,184706,False,20,...,None,None,None,None,<NA>,<NA>,<NA>,<NA>,None,None
3,2015-07-20T21:16:30Z,0vSBveAJeKBS72OIjWuipK,Take On Me (Skacoustic) - Best Of,A Best Of Us For The Rest Of Us - Bigger Bette...,03MM0GmUPrY7NvyotPa1Lz,Reel Big Fish,3bXhZFreBJF4QDUUiMmtZW,194759,False,0,...,None,None,None,None,<NA>,<NA>,<NA>,<NA>,None,None
4,2015-07-20T21:16:35Z,2OHh4ZtdL8BedYJLHP1WhK,Suckers (This One's For You),The Best Of Us For The Rest Of Us,1U7jVocQwP0iF7eQP0wmUI,Reel Big Fish,3bXhZFreBJF4QDUUiMmtZW,219560,False,0,...,None,None,None,None,<NA>,<NA>,<NA>,<NA>,None,None


## Mix 182

In [15]:

# # load some playlists
# Blink182 = get_item("Blink182", "artist", "6FBDaR13swtiWwGhX1WQsP",verbose=False)
# TheParadox = get_item("TheParadox", "artist", "6GhcI55xfZf5vqmmNqYzxW",verbose=False)
# MagnoliaPark = get_item("MagnoliaPark", "artist", "7B76SsfzG0wWk1WEvGzCmY",verbose=False)
# Sum41 = get_item("Sum41", "artist", "0qT79UgT5tY4yudH9VfsdT",verbose=False)
# All_American_Rejects = get_item("All_American_Rejects", "artist", "spotify:artist:3vAaWhdBR38Q02ohXqaNHT",verbose=False)

# dfl = [
#     Blink182,
#     TheParadox,
#     MagnoliaPark,
#     Sum41,
#     All_American_Rejects
# ]

# for temp_df in dfl:
#     temp_df = temp_df.sort_values(by=['popularity'], ascending=False).reset_index(drop=True)

# MIX182 = pd.DataFrame()

# for temp_df in dfl:
#     MIX182 = pd.concat(
#         [
#             MIX182,
#             temp_df.head(30)
#         ],
#         ignore_index=True
#     )

# MIX182 = MIX182.drop_duplicates()
# MIX182 = MIX182.sort_values(by=['popularity'], ascending=False).reset_index(drop=True)

# display(f"MIX182: {len(MIX182)}")
# display(MIX182.head())

# # save the playlist 
# playlist_id = fn.create_playlist(
#     sp,
#     name="**MIX182",
#     description="A Mix of Blink-182 and other Bands, updated via Spotify API",
#     public=True,
#     overwrite_if_exists=True
# )

# fn.add_tracks_to_playlist(sp, playlist_id, MIX182["uri"].tolist())